In [1]:
import sys
new_path = '/home/eltons-pc/Code/federated-sc-depth/src'
sys.path.append(new_path)

In [2]:
import os 
FEDERATED_TRAINING_DIRPATH = '/home/eltons-pc/Logs/federated-sc-depth/'
#FEDERATED_TRAINING_DIRPATH = '/home/eltons-pc/Backup/Logs/federated-sc-depth/'


#FEDERATED_TRAINING_ID = "15_03_2024_11:22:23" # GOOD run_ddad_to_kitti_fedscdepth_12_participants_4_per_round_3_local_epoch_iid.log
#FEDERATED_TRAINING_ID = "17_03_2024_11:34:21" # GOOD run_ddad_to_kitti_bofedscdepth_12_participants_4_per_round_3_local_epoch_iid.log
#FEDERATED_TRAINING_ID = "23_04_2024_14:12:20" # GOOD run_ddad_to_kitti_bofedscdepth_with_er_constrained_loss_12_participants_4_per_round_3_local_epoch_iid.log
FEDERATED_TRAINING_ID = "28_04_2024_23:12:02" # GOOD run_ddad_to_kitti_fedscdepth_with_er_global_retrain_12_participants_4_per_round_3_local_epoch_iid.log


FEDERATED_TRAINING_DIRPATH = os.path.join(FEDERATED_TRAINING_DIRPATH, FEDERATED_TRAINING_ID)
print('FEDERATED_TRAINING_DIRPATH', FEDERATED_TRAINING_DIRPATH)
assert os.path.exists(FEDERATED_TRAINING_DIRPATH), 'FEDERATED_TRAINING_DIRPATH does not exist!'

FEDERATED_TRAINING_DIRPATH /home/eltons-pc/Logs/federated-sc-depth/28_04_2024_23:12:02


In [3]:
import json
# Read JSON file
federated_training_state = {}
with open(os.path.join(FEDERATED_TRAINING_DIRPATH, 'federated_training_state.json'), 'r') as f:
    federated_training_state = json.load(f)

In [4]:
import plotly.graph_objects as go

# Define variables
config_args = federated_training_state['config_args']
num_participants = config_args['fed_train_num_participants']
frac_participants_per_round = config_args['fed_train_frac_participants_per_round']
num_rounds = config_args['fed_train_num_rounds']
num_participants_per_round = num_participants * frac_participants_per_round
model_size_mb = sum(list(federated_training_state["global_model_bytes_by_round"].values()))/len(federated_training_state["global_model_bytes_by_round"]) / 1024 / 1024 # measured in code
#model_size_mb = 111.417 # logged by pytorch
model_size_gb = model_size_mb / 1024
bytes_per_participant = model_size_gb * 2  # Each participant uploads the entire model to the server, and downloads the updated model

# Extract global metrics federated_training_state
global_test_loss = list(federated_training_state["global_test_loss_by_round"].values())

# Calculate communication cost
total_communication_cost = [num_participants_per_round * bytes_per_participant * round_num for round_num in range(len(global_test_loss))]

# Create global test loss figure
test_loss_fig = go.Figure()
test_loss_fig.add_trace(go.Scatter(x=list(range(len(global_test_loss))), y=global_test_loss, name="Test Loss"))
test_loss_fig.update_layout(title="Global Validation Loss per Round", xaxis_title="Round", yaxis_title="Global Validation Loss")

# Create estimated communication cost figure
communication_cost_fig = go.Figure()
communication_cost_fig.add_trace(go.Scatter(x=list(range(num_rounds)), y=total_communication_cost, name="Communication Cost"))
communication_cost_fig.update_layout(title="Communication Cost per Round", xaxis_title="Round", yaxis_title="Communication Cost (in Gigabytes)")

# Display figures
test_loss_fig.show()
communication_cost_fig.show()


ZeroDivisionError: division by zero